In [ ]:
pip install bs4 requests


In [1]:
import os
import pandas as pd
import csv
import requests
from bs4 import BeautifulSoup
import numpy as np

Quelque fonction utile 

In [2]:
def getUrl(url):
  requete = requests.get(url)
  page = requete.content
  return page


#Segmentation des elements dont nous avons besoin 

def segmentation(code):
  marqueur = code.find("sortantes")
  soup2 = code[marqueur:-1]
  soup3 = soup2.split("//", 2)
  tab1 = soup3[0].split()
  tab2 = soup3[1].split()
  return (tab1,tab2)

#Recuperation des tableau de relation sortante/entrante

def getRelation(tab1,nombre):
  sortant = list()
  for i in tab1[nombre: len(tab1)]:
    sortant.append(i.split(';')[0:6])

  return sortant 

  


In [ ]:

#obtenir tout les relation entrante ou sortant avec les nom des thermes associée 
def geTRelation2(mot1):

   url = "http://www.jeuxdemots.org/rezo-dump.php?gotermsubmit=Chercher&gotermrel=mo&rel=?gotermsubmit=Chercher&gotermrel="+'zebre'+"&rel=3Fgotermsubmit%3DChercher"
   code = str(BeautifulSoup(getUrl(url)).find("code"))
   marqueur = code.find("(Entries)")
   soup2 = code[marqueur:-1]
   soup3 = soup2.split("//",2)
   tab = soup3[0].split('\n')
   correspNomId = list()
   for i in tab[2: len(tab)]:
     correspNomId.append(i.split(';'))

   return correspNomId





In [3]:
#Fonction qui permet d'avoir le nom d'un therme a partir de son id 



#Récupération d'un tableau de relation entrante d'un mot 

def getRelationEntrante(mot):
  url = "http://www.jeuxdemots.org/rezo-dump.php?gotermsubmit=Chercher&gotermrel=mo&rel=?gotermsubmit=Chercher&gotermrel="+mot+"&rel=3Fgotermsubmit%3DChercher"
  code = str(BeautifulSoup(getUrl(url)).find("code"))
  marqueur = code.find("sortantes")
  soup2 = code[marqueur:-1]
  soup3 = soup2.split("//", 2)
  tab2 = soup3[1].split()

  #on obtient les relations entrante du mot
  entrant = getRelation(tab2,5)

  return entrant


#Récupération d'un tableau de relation sortant d'un mot

def getRelationSortante(mot):
  url = "http://www.jeuxdemots.org/rezo-dump.php?gotermsubmit=Chercher&gotermrel=mo&rel=?gotermsubmit=Chercher&gotermrel="+mot+"&rel=3Fgotermsubmit%3DChercher"
  code = str(BeautifulSoup(getUrl(url)).find("code"))
  marqueur = code.find("sortantes")
  soup2 = code[marqueur:-1]
  soup3 = soup2.split("//", 2)
  tab1 = soup3[0].split()
  

  #on obtient les relations sortante et entrante du mot
  sortant= getRelation(tab1,3)

  return sortant

#getRelationSortante('zebre')




In [4]:

#on restreint l'affichage à une relation 
#-> on renvoie les relation entrantes 
def restreintRelationEntrante(mot,relation):
  url ='http://www.jeuxdemots.org/rezo-dump.php?gotermsubmit=Chercher&gotermrel='+mot+'&rel='+ relation
  code = str(BeautifulSoup(getUrl(url)).find("code"))
  marqueur = code.find('entrantes')
  soup2 = code[marqueur:-1]
  soup3 = soup2.split("//", 2)
  tab1 = soup3[0].split()
  del tab1[0:3]
  tab1 = getRelation(tab1,3)
  return tab1
#restreintRelationEntrante('voler','r_agent-1')
 

TEST DES FONCTIONS 

In [5]:
#fonction pour récupére le nom d'un mot à partir de son id 
def correspondanceMotId( mot):
  url = "http://www.jeuxdemots.org/rezo-dump.php?gotermsubmit=Chercher&gotermrel=mo&rel=?gotermsubmit=Chercher&gotermrel="+mot+"&rel=3Fgotermsubmit%3DChercher"
  code = str(BeautifulSoup(getUrl(url)).find("code"))
  id_mot = code.split()[7].split('=')[1].split(')')[0]

  return id_mot 

#fonction qui récupére le nom d'une relation à partir de son id 
def get_id(a):

  url = "http://www.jeuxdemots.org/jdm-about-detail-relations.php"
  entite = BeautifulSoup(getUrl(url))
  name_r = list()
  for line in entite.find_all('td',{'info':'rel_name'}):
    name_r.append(line.text.replace('\t','').replace('\n','').strip())
  

  id_r =list()
  for col in entite.find_all('td',{'info':'rel_id'}):
    id_r.append(col.text)
  
  if a in name_r :
    
    return str(id_r[name_r.index(a)].split()[0])


def correspondanceIdMot(id,mot):
  url = "http://www.jeuxdemots.org/rezo-dump.php?gotermsubmit=Chercher&gotermrel=mo&rel=?gotermsubmit=Chercher&gotermrel="+mot+"&rel=3Fgotermsubmit%3DChercher"
  code = str(BeautifulSoup(getUrl(url)).find("code"))
  marqueur = code.find('(Entries)')
  soup2 = code[marqueur:-1]
  soup3 = soup2.split("//", 2)
  tab1 = soup3[0].split("\n")
  
  del tab1[0:2]
  tab1 = getRelation(tab1,0)

  for i in tab1:
    if i[1]==id:
      return str(i[2])
  
  


if correspondanceIdMot('139886','zebre') == "'franchise'":
  print('OUIIIIIIII')

correspondanceIdMot('139886','zebre')

OUIIIIIIII


"'franchise'"

INFERENCE PAR TRANSITIVITE

******************************************************************ICI C'EST PAS MAL ********************************************************



In [21]:

def isInIt(mot1, mot2,relation):
#mot1 -> mot2
  #case de base relation direct 
  id = get_id(relation)
  id_mot = correspondanceMotId(mot2)
  for i in getRelationSortante(mot1):
    if (str(i[4]) == id):
      if (str(i[3]) == id_mot):
        #print("je regarde pour" + correspondanceMotId(i[1],mot1))
        return True
        
def fonction(mot1,mot2,relation):
  if (isInIt(mot1,mot2,relation)): 
    return mot1+" " + relation + " " +mot2

  else :
    #1->2 2->3
    for i in getRelationSortante(mot1):
      #on prends les mot sortant et on regarde si ils sont en relation avec mot2
      #print("on regarde"+ correspondanceMotId(i[1],mot1)+"->"+mot2)
      #on change le mot1 on regarde si il est en relation avec le mot2
      fonction(str(i[1]),mot2,relation)

#fonction('Tour Eiffel', 'France','15')
#piston r_holo voiture 
fonction('piston', 'voiture', 'r_holo')


#relation direct : ok !
#transitivité : ? 

#getRelationSortante('piston')

'piston r_holo voiture'

INFERENCE DEDUCTIVE

In [ ]:
INFERENCE INDUCTIVE
# pigeon r_agent-1 voler ⇒ oui car pigeon r_isa oiseau et oiseau r_agent-1 voler
#on doit faire un sorte de chainage arriére 

id = get_id('r_agent-1')
for i in restreintRelationEntrante('voler','r_agent-1'):
  if j in getRelationSortante(correspondanceIdMot(i[2],'voler'):
    if  ( i[]== 'pigeon' ): 
    print(str(i[2] + +'pigeon'))
   


On doit pouvoir donner en entrée deux therme et une relation et renvoyer un chemin peut importe si c'est vrai ou faux ! 

In [ ]:
print("Vueillez entrée dans l'ordre un therme, une relation et enfin un therme ! ")
a = input()
therme1 = a.split()[0]
therme2 = a.split()[1]
relation = a.split()[2]


print("On veut savoir si " + therme1 + get_id(relation) + therme2)
url1 = "http://www.jeuxdemots.org/rezo-dump.php?gotermsubmit=Chercher&gotermrel=mo&rel=?gotermsubmit=Chercher&gotermrel="+therme1+"&rel=3Fgotermsubmit%3DChercher"
url2 = "http://www.jeuxdemots.org/rezo-dump.php?gotermsubmit=Chercher&gotermrel=mo&rel=?gotermsubmit=Chercher&gotermrel="+therme2+"&rel=3Fgotermsubmit%3DChercher"

code1 = str( BeautifulSoup(getUrl(url1)).find("code"))
code2 = str(BeautifulSoup(getUrl(url2)).find("code"))


#on obtient l'id des mots 
id_mot1 = code1.split()[7].split('=')[1].split(')')[0]
id_mot2 = code2.split()[7].split('=')[1].split(')')[0]


#on obtient les relations sortante et entrante du mot
sortant1 = getRelation(segmentation(code1)[0],3)
entrant1 = getRelation(segmentation(code1)[1],5)

sortant2 = getRelation(segmentation(code2)[0],3)
entrant2 = getRelation(segmentation(code2)[1],5)





entrant1



Vueillez entrée dans l'ordre un therme, une relation et enfin un therme ! 


KeyboardInterrupt: ignored

In [ ]:
a= input()
print("entréé" + a.split()[1] )


coucou la famille
entrééla


In [ ]:
#sorte de chainage avant 

Ecriture dans un fichier texte

In [ ]:
f = open(a+'.txt','w',encoding="utf8")
b = str(code)
f.write(b)

4

In [ ]:
with open(a+".txt", 'r') as filin:
    lignes = filin.readlines()
    for ligne in lignes:
        if ligne  :
          #print(ligne)

SyntaxError: ignored